In [41]:
from mnist import MNIST
from scipy.stats import norm
import numpy as np

## Read train and test data

In [34]:
mndata = MNIST('data/mnist_data_files')
mndata.gz=True
train_images, train_labels = mndata.load_training()

In [35]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [37]:
# 60000 rows 28*28 pixels
print(train_images.shape) 
print(train_labels.shape)

(60000, 784)
(60000,)


In [38]:
test_images, test_labels = mndata.load_testing()

In [39]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [44]:
print(test_images.shape)
print(test_labels.shape)

(10000, 784)
(10000,)


In [46]:
features = train_images.T

In [47]:
features.shape

(784, 60000)

In [53]:
norm.fit([[1,1], [1,1]])

(1.0, 0.0)

## Naive Bayes - normal distribution

In [ ]:
def get_class_feature_summary():
    
def get_accuracy_for_one_iteration(distribution):
    if(distribution=='nd'):
        summary = get_class_feature_summary(train_set, ignore_missing_value)
        predicts = test_set.apply(lambda x: predict(summary, x[:8]), axis=1)
        accuracy = calculate_accuracy(test_set.Class.tolist(), predicts.tolist())
        return accuracy

In [ ]:
accuracy = get_accuracy_for_one_iteration('nd')